# Korean & English Data Extraction

This notebook extracts Korean and English text from multiple sources for building a bilingual neural sparse model.

**Data Sources:**
1. **Korean Wikipedia** - Encyclopedia articles (direct XML dumps from Wikimedia)
2. **English Wikipedia** - Encyclopedia articles (direct XML dumps from Wikimedia)
3. **NamuWiki** - Korean wiki encyclopedia (via HuggingFace: `heegyu/namuwiki-extracted`)
4. **모두의 말뭉치** - Korean corpus from National Institute of Korean Language (via Korpora library)

**Updated**: November 2025 - Using latest data dumps for all sources

## Steps
1. Load data from all sources
2. Parse and extract text
3. Clean and filter content
4. Save processed data in chunks (50K items per file)

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../..')

from src.data.wikipedia_xml_parser import WikipediaXMLParser
from src.data.namuwiki_parser import NamuWikiParser
from src.data.modu_corpus_parser import ModuCorpusParser
from pathlib import Path
import json

## 1. Setup Paths

In [2]:
# Output directory
output_dir = Path("../../dataset/wikipedia")
output_dir.mkdir(parents=True, exist_ok=True)

# We'll split files into chunks to avoid very large files
ARTICLES_PER_FILE = 50000  # 50K articles per file

# Processing control
SKIP_IF_EXISTS = True  # Set to False to force re-processing

## 2. Extract Korean Wikipedia Articles

**Processing all Korean Wikipedia articles** (no limit)

Files will be saved in chunks of 50,000 articles each to avoid very large files.

**Note**: First run will download the Wikipedia dump (~GB size). Subsequent runs will use cached file.

In [3]:
import glob
import os

# Check if Korean chunk files already exist
ko_existing_chunks = sorted(glob.glob(str(output_dir / "ko_articles_chunk_*.jsonl")))

if SKIP_IF_EXISTS and ko_existing_chunks:
    print("=" * 80)
    print("✓ Korean Wikipedia chunk files already exist!")
    print("=" * 80)
    print(f"\nFound {len(ko_existing_chunks)} existing chunk files:")
    
    ko_articles_total = []
    for chunk_file in ko_existing_chunks:
        chunk_articles = []
        with open(chunk_file, 'r', encoding='utf-8') as f:
            for line in f:
                chunk_articles.append(json.loads(line))
        ko_articles_total.extend(chunk_articles)
        print(f"  - {Path(chunk_file).name}: {len(chunk_articles):,} articles")
    
    print(f"\nTotal: {len(ko_articles_total):,} Korean articles loaded from cache")
    print("\n💡 Set SKIP_IF_EXISTS = False to force re-processing")
    
else:
    # Initialize Korean parser (using latest dump)
    ko_parser = WikipediaXMLParser(
        language="ko",
        date="latest",  # Will automatically use the most recent dump
        cache_dir="../../dataset/wikipedia/cache"
    )

    # Check if XML dump already exists in cache
    if ko_parser.date == "latest":
        ko_parser.date = ko_parser.get_latest_dump_date()
    
    dump_path = ko_parser.cache_dir / f"{ko_parser.language}wiki-{ko_parser.date}.xml.bz2"
    
    if dump_path.exists():
        print("=" * 80)
        print(f"✓ Wikipedia XML dump already cached: {dump_path.name}")
        print(f"  Size: {os.path.getsize(dump_path) / 1024 / 1024 / 1024:.2f} GB")
        print("=" * 80)
    else:
        print("=" * 80)
        print("⬇ Downloading Wikipedia XML dump (this will take a while)...")
        print("=" * 80)
    
    # Download the dump (will skip if already exists)
    dump_path = ko_parser.download_dump()

    print("\n" + "="*80)
    print("Processing ALL Korean Wikipedia articles")
    print("Files will be split into chunks of 50,000 articles")
    print("="*80 + "\n")

    # Process articles in streaming mode and save in chunks
    from tqdm import tqdm

    ko_articles_total = []
    chunk_num = 0
    current_chunk = []

    iterator = ko_parser.iter_articles(dump_path)
    pbar = tqdm(iterator, desc="Processing Korean Wikipedia")

    for raw_article in pbar:
        # Parse wikitext to plain text
        text = ko_parser.parse_wikitext(raw_article["wikitext"])
        
        article = {
            "id": raw_article["id"],
            "url": raw_article["url"],
            "title": raw_article["title"],
            "text": text,
            "language": "ko",
        }
        
        # Apply filters
        if ko_parser.filter_article(article, min_length=200, max_length=100000):
            current_chunk.append(article)
            ko_articles_total.append(article)
            
            # Save chunk when it reaches the limit
            if len(current_chunk) >= ARTICLES_PER_FILE:
                chunk_num += 1
                output_file = output_dir / f"ko_articles_chunk_{chunk_num:03d}.jsonl"
                ko_parser.save_articles(current_chunk, output_file)
                pbar.set_postfix({
                    'chunks': chunk_num, 
                    'articles': len(ko_articles_total),
                    'current_chunk': len(current_chunk)
                })
                current_chunk = []

    # Save remaining articles in last chunk
    if current_chunk:
        chunk_num += 1
        output_file = output_dir / f"ko_articles_chunk_{chunk_num:03d}.jsonl"
        ko_parser.save_articles(current_chunk, output_file)

    print(f"\n✓ Processed {len(ko_articles_total):,} Korean articles")
    print(f"✓ Saved in {chunk_num} chunk files")
    if ko_articles_total:
        print(f"✓ Sample article: {ko_articles_total[0]['title']}")

✓ Wikipedia XML dump already cached: kowiki-20251101.xml.bz2
  Size: 1.18 GB
Using cached dump: ../../dataset/wikipedia/cache/kowiki-20251101.xml.bz2

Processing ALL Korean Wikipedia articles
Files will be split into chunks of 50,000 articles



Processing Korean Wikipedia: 5it [00:00, 34.74it/s]

Parsing Wikipedia dump: ../../dataset/wikipedia/cache/kowiki-20251101.xml.bz2


Processing Korean Wikipedia: 55918it [03:37, 92.89it/s, chunks=1, articles=5e+4, current_chunk=5e+4] 

Saved 50000 articles to ../../dataset/wikipedia/ko_articles_chunk_001.jsonl


...

Saved 50000 articles to ../../dataset/wikipedia/ko_articles_chunk_010.jsonl


Processing Korean Wikipedia: 711968it [24:30, 103.70it/s, chunks=11, articles=550000, current_chunk=5e+4]

Saved 50000 articles to ../../dataset/wikipedia/ko_articles_chunk_011.jsonl


Processing Korean Wikipedia: 726340it [25:25, 476.17it/s, chunks=11, articles=550000, current_chunk=5e+4]


Saved 11999 articles to ../../dataset/wikipedia/ko_articles_chunk_012.jsonl

✓ Processed 561,999 Korean articles
✓ Saved in 12 chunk files
✓ Sample article: 지미 카터


## 3. Extract English Wikipedia Articles

**Processing all English Wikipedia articles** (no limit)

Files will be saved in chunks of 50,000 articles each.

In [ ]:
# Check if English chunk files already exist
en_existing_chunks = sorted(glob.glob(str(output_dir / "en_articles_chunk_*.jsonl")))

if SKIP_IF_EXISTS and en_existing_chunks:
    print("=" * 80)
    print("✓ English Wikipedia chunk files already exist!")
    print("=" * 80)
    print(f"\nFound {len(en_existing_chunks)} existing chunk files:")
    
    en_articles_total = []
    for chunk_file in en_existing_chunks:
        chunk_articles = []
        with open(chunk_file, 'r', encoding='utf-8') as f:
            for line in f:
                chunk_articles.append(json.loads(line))
        en_articles_total.extend(chunk_articles)
        print(f"  - {Path(chunk_file).name}: {len(chunk_articles):,} articles")
    
    print(f"\nTotal: {len(en_articles_total):,} English articles loaded from cache")
    print("\n💡 Set SKIP_IF_EXISTS = False to force re-processing")
    
else:
    # Initialize English parser (using latest dump)
    en_parser = WikipediaXMLParser(
        language="en",
        date="latest",  # Will automatically use the most recent dump
        cache_dir="../../dataset/wikipedia/cache"
    )

    # Check if XML dump already exists in cache
    if en_parser.date == "latest":
        en_parser.date = en_parser.get_latest_dump_date()
    
    dump_path = en_parser.cache_dir / f"{en_parser.language}wiki-{en_parser.date}.xml.bz2"
    
    if dump_path.exists():
        print("=" * 80)
        print(f"✓ Wikipedia XML dump already cached: {dump_path.name}")
        print(f"  Size: {os.path.getsize(dump_path) / 1024 / 1024 / 1024:.2f} GB")
        print("=" * 80)
    else:
        print("=" * 80)
        print("⬇ Downloading Wikipedia XML dump (this will take a while)...")
        print("=" * 80)
    
    # Download the dump (will skip if already exists)
    dump_path = en_parser.download_dump()

    print("\n" + "="*80)
    print("Processing ALL English Wikipedia articles")
    print("Files will be split into chunks of 50,000 articles")
    print("="*80 + "\n")

    # Process articles in streaming mode and save in chunks
    en_articles_total = []
    chunk_num = 0
    current_chunk = []

    iterator = en_parser.iter_articles(dump_path)
    pbar = tqdm(iterator, desc="Processing English Wikipedia")

    for raw_article in pbar:
        # Parse wikitext to plain text
        text = en_parser.parse_wikitext(raw_article["wikitext"])
        
        article = {
            "id": raw_article["id"],
            "url": raw_article["url"],
            "title": raw_article["title"],
            "text": text,
            "language": "en",
        }
        
        # Apply filters
        if en_parser.filter_article(article, min_length=200, max_length=100000):
            current_chunk.append(article)
            en_articles_total.append(article)
            
            # Save chunk when it reaches the limit
            if len(current_chunk) >= ARTICLES_PER_FILE:
                chunk_num += 1
                output_file = output_dir / f"en_articles_chunk_{chunk_num:03d}.jsonl"
                en_parser.save_articles(current_chunk, output_file)
                pbar.set_postfix({
                    'chunks': chunk_num, 
                    'articles': len(en_articles_total),
                    'current_chunk': len(current_chunk)
                })
                current_chunk = []

    # Save remaining articles in last chunk
    if current_chunk:
        chunk_num += 1
        output_file = output_dir / f"en_articles_chunk_{chunk_num:03d}.jsonl"
        en_parser.save_articles(current_chunk, output_file)

    print(f"\n✓ Processed {len(en_articles_total):,} English articles")
    print(f"✓ Saved in {chunk_num} chunk files")
    if en_articles_total:
        print(f"✓ Sample article: {en_articles_total[0]['title']}")

✓ Wikipedia XML dump already cached: enwiki-20251101.xml.bz2
  Size: 23.95 GB
Using cached dump: ../../dataset/wikipedia/cache/enwiki-20251101.xml.bz2

Processing ALL English Wikipedia articles
Files will be split into chunks of 50,000 articles



Processing English Wikipedia: 4it [00:00, 32.85it/s]

Parsing Wikipedia dump: ../../dataset/wikipedia/cache/enwiki-20251101.xml.bz2


Processing English Wikipedia: 52133it [14:24, 13.99it/s, chunks=1, articles=5e+4, current_chunk=5e+4] 

Saved 50000 articles to ../../dataset/wikipedia/en_articles_chunk_001.jsonl


...

Processing English Wikipedia: 3789090it [5:35:29, 493.66it/s, chunks=71, articles=3550000, current_chunk=5e+4] 

Saved 50000 articles to ../../dataset/wikipedia/en_articles_chunk_071.jsonl


Processing English Wikipedia: 3842265it [5:38:46, 149.42it/s, chunks=72, articles=3.6e+6, current_chunk=5e+4]  

Saved 50000 articles to ../../dataset/wikipedia/en_articles_chunk_072.jsonl


Processing English Wikipedia: 3864632it [5:40:12, 296.09it/s, chunks=72, articles=3.6e+6, current_chunk=5e+4]

## 4. Extract NamuWiki Articles (Korean)

**NamuWiki** is a Korean wiki encyclopedia with ~1.5M articles, providing additional Korean language data.

Using HuggingFace dataset: `heegyu/namuwiki-extracted`

Files will be saved in chunks of 50,000 articles each.

In [ ]:
# Check if NamuWiki chunk files already exist
namu_existing_chunks = sorted(glob.glob(str(output_dir / "../namuwiki/namuwiki_chunk_*.jsonl")))

if SKIP_IF_EXISTS and namu_existing_chunks:
    print("=" * 80)
    print("✓ NamuWiki chunk files already exist!")
    print("=" * 80)
    print(f"\nFound {len(namu_existing_chunks)} existing chunk files:")
    
    namu_articles_total = []
    for chunk_file in namu_existing_chunks:
        chunk_articles = []
        with open(chunk_file, 'r', encoding='utf-8') as f:
            for line in f:
                chunk_articles.append(json.loads(line))
        namu_articles_total.extend(chunk_articles)
        print(f"  - {Path(chunk_file).name}: {len(chunk_articles):,} articles")
    
    print(f"\nTotal: {len(namu_articles_total):,} NamuWiki articles loaded from cache")
    print("\n💡 Set SKIP_IF_EXISTS = False to force re-processing")
    
else:
    # Initialize NamuWiki parser
    namu_parser = NamuWikiParser(cache_dir="../../dataset/namuwiki/cache")
    
    print("=" * 80)
    print("Processing NamuWiki articles from HuggingFace")
    print("Files will be split into chunks of 50,000 articles")
    print("=" * 80)
    print("\n⬇ This will download ~2GB of data on first run...")
    
    # Process NamuWiki articles (will automatically chunk the output)
    namu_articles_total = namu_parser.process_namuwiki(
        output_path="../../dataset/namuwiki/namuwiki_articles.jsonl",
        max_articles=None,  # Process all articles
        min_length=100,
        max_length=100000,
        chunk_size=ARTICLES_PER_FILE
    )
    
    print(f"\n✓ Processed {len(namu_articles_total):,} NamuWiki articles")
    if namu_articles_total:
        print(f"✓ Sample article: {namu_articles_total[0]['title']}")

## 5. Extract 모두의 말뭉치 (Korean)

**모두의 말뭉치** (Everyone's Corpus) is a large-scale Korean language corpus from the National Institute of Korean Language.

Includes:
- News articles (신문 말뭉치)
- Spoken language (구어 말뭉치)
- Web text (웹 말뭉치)
- Messenger conversations (메신저 말뭉치)

Files will be saved in chunks of 50,000 texts each.

**Note**: Some corpora may require authentication. The parser will automatically skip unavailable corpora.

## 6. Inspect Sample Articles


In [ ]:
# Display Korean article sample
if len(ko_articles_total) > 0:
    # Use first available article or 10th if available
    sample_idx = min(10, len(ko_articles_total) - 1)
    sample_ko = ko_articles_total[sample_idx]
    
    print("=" * 80)
    print(f"Article #{sample_idx + 1} of {len(ko_articles_total):,}")
    print(f"Title: {sample_ko['title']}")
    print(f"URL: {sample_ko['url']}")
    print(f"Language: {sample_ko['language']}")
    print(f"Text length: {len(sample_ko['text'])} characters")
    print("\nFirst 300 characters:")
    print(sample_ko['text'][:300])
    print("=" * 80)
else:
    print("No articles found. Check filtering criteria.")

## 7. Statistics

In [ ]:
import numpy as np

print("=" * 80)
print("DATA EXTRACTION STATISTICS")
print("=" * 80)

# Korean Wikipedia stats
if len(ko_articles_total) > 0:
    ko_lengths = [len(a['text']) for a in ko_articles_total]
    print("\n1. Korean Wikipedia:")
    print(f"     Total: {len(ko_articles_total):,}")
    print(f"     Mean length: {np.mean(ko_lengths):.0f} chars")
    print(f"     Median length: {np.median(ko_lengths):.0f} chars")
    print(f"     Min length: {np.min(ko_lengths):.0f} chars")
    print(f"     Max length: {np.max(ko_lengths):.0f} chars")
else:
    print("\n1. Korean Wikipedia: No articles found")

# English Wikipedia stats
if len(en_articles_total) > 0:
    en_lengths = [len(a['text']) for a in en_articles_total]
    print("\n2. English Wikipedia:")
    print(f"     Total: {len(en_articles_total):,}")
    print(f"     Mean length: {np.mean(en_lengths):.0f} chars")
    print(f"     Median length: {np.median(en_lengths):.0f} chars")
    print(f"     Min length: {np.min(en_lengths):.0f} chars")
    print(f"     Max length: {np.max(en_lengths):.0f} chars")
else:
    print("\n2. English Wikipedia: No articles found")

# NamuWiki stats
if 'namu_articles_total' in locals() and len(namu_articles_total) > 0:
    namu_lengths = [len(a['text']) for a in namu_articles_total]
    print("\n3. NamuWiki (Korean):")
    print(f"     Total: {len(namu_articles_total):,}")
    print(f"     Mean length: {np.mean(namu_lengths):.0f} chars")
    print(f"     Median length: {np.median(namu_lengths):.0f} chars")
    print(f"     Min length: {np.min(namu_lengths):.0f} chars")
    print(f"     Max length: {np.max(namu_lengths):.0f} chars")
else:
    print("\n3. NamuWiki: No articles found")

# 모두의 말뭉치 stats
if 'modu_articles_total' in locals() and len(modu_articles_total) > 0:
    modu_lengths = [len(a['text']) for a in modu_articles_total]
    print("\n4. 모두의 말뭉치 (Korean):")
    print(f"     Total: {len(modu_articles_total):,}")
    print(f"     Mean length: {np.mean(modu_lengths):.0f} chars")
    print(f"     Median length: {np.median(modu_lengths):.0f} chars")
    print(f"     Min length: {np.min(modu_lengths):.0f} chars")
    print(f"     Max length: {np.max(modu_lengths):.0f} chars")
else:
    print("\n4. 모두의 말뭉치: No texts found")

# Total statistics
print("\n" + "=" * 80)
print("LANGUAGE BALANCE:")
print("=" * 80)

korean_total = 0
english_total = 0

if len(ko_articles_total) > 0:
    korean_total += len(ko_articles_total)
if 'namu_articles_total' in locals() and len(namu_articles_total) > 0:
    korean_total += len(namu_articles_total)
if 'modu_articles_total' in locals() and len(modu_articles_total) > 0:
    korean_total += len(modu_articles_total)
if len(en_articles_total) > 0:
    english_total += len(en_articles_total)

print(f"\nKorean texts:  {korean_total:,}")
print(f"English texts: {english_total:,}")
print(f"Total texts:   {korean_total + english_total:,}")

if korean_total > 0 and english_total > 0:
    ratio = english_total / korean_total
    print(f"\nEnglish/Korean ratio: {ratio:.2f}x")
    
print("=" * 80)

## 8. Verify Saved Files

In [ ]:
import os
import glob

print("=" * 80)
print("SAVED CHUNK FILES")
print("=" * 80)

# Find all Korean Wikipedia chunk files
ko_chunks = sorted(glob.glob(str(output_dir / "ko_articles_chunk_*.jsonl")))
if ko_chunks:
    print(f"\n1. Korean Wikipedia: {len(ko_chunks)} chunk files")
    total_size = sum(os.path.getsize(f) for f in ko_chunks)
    total_lines = sum(sum(1 for _ in open(f)) for f in ko_chunks)
    print(f"   Total size: {total_size / 1024 / 1024:.2f} MB")
    print(f"   Total articles: {total_lines:,}")
    print(f"   Files:")
    for chunk in ko_chunks[:3]:  # Show first 3
        size = os.path.getsize(chunk) / 1024 / 1024
        lines = sum(1 for _ in open(chunk))
        print(f"     - {os.path.basename(chunk):30s} ({size:>6.2f} MB, {lines:>6,} articles)")
    if len(ko_chunks) > 3:
        print(f"     ... and {len(ko_chunks) - 3} more files")
else:
    print("\n1. Korean Wikipedia: No chunk files found")

# Find all English Wikipedia chunk files
en_chunks = sorted(glob.glob(str(output_dir / "en_articles_chunk_*.jsonl")))
if en_chunks:
    print(f"\n2. English Wikipedia: {len(en_chunks)} chunk files")
    total_size = sum(os.path.getsize(f) for f in en_chunks)
    total_lines = sum(sum(1 for _ in open(f)) for f in en_chunks)
    print(f"   Total size: {total_size / 1024 / 1024:.2f} MB")
    print(f"   Total articles: {total_lines:,}")
    print(f"   Files:")
    for chunk in en_chunks[:3]:  # Show first 3
        size = os.path.getsize(chunk) / 1024 / 1024
        lines = sum(1 for _ in open(chunk))
        print(f"     - {os.path.basename(chunk):30s} ({size:>6.2f} MB, {lines:>6,} articles)")
    if len(en_chunks) > 3:
        print(f"     ... and {len(en_chunks) - 3} more files")
else:
    print("\n2. English Wikipedia: No chunk files found")

# Find all NamuWiki chunk files
namu_dir = output_dir / "../namuwiki"
namu_chunks = sorted(glob.glob(str(namu_dir / "namuwiki_chunk_*.jsonl")))
if namu_chunks:
    print(f"\n3. NamuWiki (Korean): {len(namu_chunks)} chunk files")
    total_size = sum(os.path.getsize(f) for f in namu_chunks)
    total_lines = sum(sum(1 for _ in open(f)) for f in namu_chunks)
    print(f"   Total size: {total_size / 1024 / 1024:.2f} MB")
    print(f"   Total articles: {total_lines:,}")
    print(f"   Files:")
    for chunk in namu_chunks[:3]:  # Show first 3
        size = os.path.getsize(chunk) / 1024 / 1024
        lines = sum(1 for _ in open(chunk))
        print(f"     - {os.path.basename(chunk):30s} ({size:>6.2f} MB, {lines:>6,} articles)")
    if len(namu_chunks) > 3:
        print(f"     ... and {len(namu_chunks) - 3} more files")
else:
    print("\n3. NamuWiki: No chunk files found")

# Find all 모두의 말뭉치 chunk files
modu_dir = output_dir / "../modu"
modu_chunks = sorted(glob.glob(str(modu_dir / "modu_chunk_*.jsonl")))
if modu_chunks:
    print(f"\n4. 모두의 말뭉치 (Korean): {len(modu_chunks)} chunk files")
    total_size = sum(os.path.getsize(f) for f in modu_chunks)
    total_lines = sum(sum(1 for _ in open(f)) for f in modu_chunks)
    print(f"   Total size: {total_size / 1024 / 1024:.2f} MB")
    print(f"   Total texts: {total_lines:,}")
    print(f"   Files:")
    for chunk in modu_chunks[:3]:  # Show first 3
        size = os.path.getsize(chunk) / 1024 / 1024
        lines = sum(1 for _ in open(chunk))
        print(f"     - {os.path.basename(chunk):30s} ({size:>6.2f} MB, {lines:>6,} texts)")
    if len(modu_chunks) > 3:
        print(f"     ... and {len(modu_chunks) - 3} more files")
else:
    print("\n4. 모두의 말뭉치: No chunk files found")

print("\n" + "=" * 80)

## Summary

We've successfully extracted and cleaned **ALL** articles from multiple Korean and English data sources.

**Data Sources:**
1. **Korean Wikipedia** - Encyclopedia articles (~600K articles)
2. **English Wikipedia** - Encyclopedia articles (~6M articles)
3. **NamuWiki** - Korean wiki encyclopedia (~1.5M articles)
4. **모두의 말뭉치** - Korean language corpus from National Institute of Korean Language
   - News articles (신문 말뭉치)
   - Spoken language (구어 말뭉치)
   - Web text (웹 말뭉치)
   - Messenger conversations (메신저 말뭉치)

**Key Features:**
- ✅ Processes complete datasets (no article limit)
- ✅ Saves data in manageable chunks (50,000 items per file)
- ✅ Filters out redirects, special pages, and low-quality content
- ✅ Cleans markup to plain text
- ✅ Implements 2-level caching (chunk files + raw dumps)
- ✅ Significantly improved Korean-English data balance

**Output Structure:**
```
dataset/
├── wikipedia/
│   ├── ko_articles_chunk_001.jsonl
│   ├── ko_articles_chunk_002.jsonl
│   ├── ...
│   ├── en_articles_chunk_001.jsonl
│   ├── en_articles_chunk_002.jsonl
│   └── ...
├── namuwiki/
│   ├── namuwiki_chunk_001.jsonl
│   ├── namuwiki_chunk_002.jsonl
│   └── ...
└── modu/
    ├── modu_chunk_001.jsonl
    ├── modu_chunk_002.jsonl
    └── ...
```

**Next steps:**
- Extract inter-language links from chunks
- Extract synonym pairs from article text
- Build comprehensive bilingual dictionary
- Train neural sparse model with balanced Korean-English data

## Summary

We've successfully extracted and cleaned **ALL** Korean and English Wikipedia articles.

**Key Features:**
- ✅ Processes complete Wikipedia dumps (no article limit)
- ✅ Saves data in manageable chunks (50,000 articles per file)
- ✅ Filters out redirects, special pages, and low-quality articles
- ✅ Cleans MediaWiki markup to plain text
- ✅ Ready for synonym extraction and model training

**Output Structure:**
```
dataset/wikipedia/
├── ko_articles_chunk_001.jsonl  (50,000 articles)
├── ko_articles_chunk_002.jsonl  (50,000 articles)
├── ...
├── en_articles_chunk_001.jsonl  (50,000 articles)
├── en_articles_chunk_002.jsonl  (50,000 articles)
└── ...
```

**Next steps:**
- Extract inter-language links from chunks
- Extract synonym pairs from article text
- Build comprehensive bilingual dictionary